In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d

In [ ]:
point_cloud = np.loadtxt('sample_w_normals.xyz',skiprows=1)           # with normals

In [ ]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(point_cloud[:,:3])
pcd.colors = o3d.utility.Vector3dVector(point_cloud[:,3:6]/255)
pcd.normals = o3d.utility.Vector3dVector(point_cloud[:,6:9])

In [ ]:
# for visualization
o3d.visualization.draw_geometries([pcd])

### strategy 1: Ball Pivoting Algorithm

In [ ]:
distances = pcd.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
radius = 3 * avg_dist

In [ ]:
bpa_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd,o3d.utility.DoubleVector([radius, radius * 2]))

In [ ]:
dec_mesh = bpa_mesh.simplify_quadric_decimation(100000)

In [ ]:
dec_mesh.remove_degenerate_triangles()
dec_mesh.remove_duplicated_triangles()
dec_mesh.remove_duplicated_vertices()
dec_mesh.remove_non_manifold_edges()

In [ ]:
o3d.io.write_triangle_mesh("bpa_mesh.ply", dec_mesh)

In [ ]:
# provo a esportarla in .msh per aprirla in gmsh -> no
o3d.io.write_triangle_mesh("bpa_mesh.msh", dec_mesh)

In [ ]:
def lod_mesh_export(mesh, lods, extension):
    mesh_lods={}
    for i in lods:
        mesh_lod = mesh.simplify_quadric_decimation(i)
        o3d.io.write_triangle_mesh("bpa_lod_"+str(i)+extension, mesh_lod)
        mesh_lods[i]=mesh_lod
    print("generation of "+str(i)+" LoD successful")
    return mesh_lods

In [ ]:
my_lods = lod_mesh_export(bpa_mesh, [100000,50000,10000,1000,100], ".ply")

In [ ]:
o3d.visualization.draw_geometries([my_lods[10000]])

### strategy 2: poisson reconstruction

In [ ]:
poisson_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=8, width=0, scale=1.1, linear_fit=False)[0]

In [ ]:
bbox = pcd.get_axis_aligned_bounding_box()
p_mesh_crop = poisson_mesh.crop(bbox)

In [ ]:
o3d.io.write_triangle_mesh("p_mesh_c.ply", p_mesh_crop)

In [ ]:
def lod_mesh_export_pois(mesh, lods, extension):
    mesh_lods={}
    for i in lods:
        mesh_lod = mesh.simplify_quadric_decimation(i)
        o3d.io.write_triangle_mesh("pois_lod_"+str(i)+extension, mesh_lod)
        mesh_lods[i]=mesh_lod
    print("generation of "+str(i)+" LoD successful")
    return mesh_lods

In [ ]:
my_lods = lod_mesh_export_pois(p_mesh_crop, [100000,50000,10000,1000,100], ".ply")


In [ ]:
o3d.visualization.draw_geometries([my_lods[1000]])